In [35]:
from PIL import Image
import numpy as np


def fname_to_vgg_input(fname):
    """ Creates the input for a VGG network from the filename 
        Args: 
            fname (string) : the filename to be parsed
        Returns: 
            (numpy ndarray) : the array to be passed into the VGG network as a single example
    """
    im = Image.open(fname)
    if im.mode != "RGB":
        im = im.convert(mode="RGB")
    return np.rollaxis(np.asarray(im),2)


thing = fname_to_vgg_input("female_cropped_normalized/30598825034_crop1_normalized.jpg")


In [6]:
import keras
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD

def VGG_16(weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='softmax'))

    if weights_path:
        model.load_weights(weights_path)
        
    for i in xrange(5):
        model.pop()

    model2 = Sequential()
    model2.add(ZeroPadding2D((1,1),input_shape=(3,75,75)))
    model2.add(Convolution2D(64, 3, 3, activation='relu'))
    model2.add(ZeroPadding2D((1,1)))
    model2.add(Convolution2D(64, 3, 3, activation='relu'))
    model2.add(MaxPooling2D((2,2), strides=(2,2)))

    model2.add(ZeroPadding2D((1,1)))
    model2.add(Convolution2D(128, 3, 3, activation='relu'))
    model2.add(ZeroPadding2D((1,1)))
    model2.add(Convolution2D(128, 3, 3, activation='relu'))
    model2.add(MaxPooling2D((2,2), strides=(2,2)))

    model2.add(ZeroPadding2D((1,1)))
    model2.add(Convolution2D(256, 3, 3, activation='relu'))
    model2.add(ZeroPadding2D((1,1)))
    model2.add(Convolution2D(256, 3, 3, activation='relu'))
    model2.add(ZeroPadding2D((1,1)))
    model2.add(Convolution2D(256, 3, 3, activation='relu'))
    model2.add(MaxPooling2D((2,2), strides=(2,2)))

    model2.add(ZeroPadding2D((1,1)))
    model2.add(Convolution2D(512, 3, 3, activation='relu'))
    model2.add(ZeroPadding2D((1,1)))
    model2.add(Convolution2D(512, 3, 3, activation='relu'))
    model2.add(ZeroPadding2D((1,1)))
    model2.add(Convolution2D(512, 3, 3, activation='relu'))
    model2.add(MaxPooling2D((2,2), strides=(2,2)))

    model2.add(ZeroPadding2D((1,1)))
    model2.add(Convolution2D(512, 3, 3, activation='relu'))
    model2.add(ZeroPadding2D((1,1)))
    model2.add(Convolution2D(512, 3, 3, activation='relu'))
    model2.add(ZeroPadding2D((1,1)))
    model2.add(Convolution2D(512, 3, 3, activation='relu'))
    model2.add(MaxPooling2D((2,2), strides=(2,2)))

    model2.add(Flatten())
        
    weights = model.get_weights()
    #print weights
    model2.set_weights(weights)
        
    return model2
    
# AUTOLAB_IGNORE_START
truncated_vgg = VGG_16("vgg16_weights.h5")
print truncated_vgg
# AUTOLAB_IGNORE_STOP

In [40]:
import os
import copy

females = map(lambda path: "female_cropped_normalized/"+path, os.listdir("female_cropped_normalized"))
males = map(lambda path: "male_cropped_normalized/"+path, os.listdir("male_cropped_normalized"))
files = copy.copy(females)
files.extend(males)
files = filter(lambda path: path.find("jpg") > -1, files)



In [41]:
def fnames_to_features(fnames, vgg):
    """ Given a list of filenames and a VGG16 model, generate the corresponding array of VGG features
        Args: 
            fnames (list) : A list of filenames
            vgg (keras model) : a pretrained VGG16 model
        Returns: 
            (numpy ndarray) : a 2D array of features generated from the VGG model
    """
    vgg.compile(optimizer="adam", loss='categorical_crossentropy')
    vgg_input = map(lambda fname: fname_to_vgg_input(fname), fnames)

    return vgg.predict(np.array(vgg_input))

inp = files

thing2 = fnames_to_features(inp, truncated_vgg)

np.savetxt("features.txt", thing2)
labels = map(lambda path: path.find("female")> -1, inp)
np.savetxt("labels.txt", np.asarray(labels))
    

In [48]:
features = np.loadtxt("features.txt")
labels = np.loadtxt("labels.txt", dtype=float).astype(int)

In [54]:
import time

def predict_from_features(X, y, X_te):
    from sklearn.linear_model import SGDClassifier
    clf = SGDClassifier(loss="log", penalty="l1")
    clf.fit(X, y)
    return clf.predict(X_te)

perm = np.random.permutation(len(labels))

numberToTrainOn = int(len(labels)*0.8)
training_features = features[perm[:numberToTrainOn]]
training_labels = labels[perm[:numberToTrainOn]]
testing_features = features[perm[numberToTrainOn:]]
testing_labels = labels[perm[numberToTrainOn:]]

start = time.time()
y_p = predict_from_features(training_features, training_labels, testing_features)
end = time.time()
print "Validation accuracy: {} in {} seconds".format(np.mean(y_p==testing_labels), end-start)

Validation accuracy: 0.836795252226 in 0.102756977081 seconds
